In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install hyperopt
!ls -a /kaggle/input/jane-street-market-prediction/train.csv


In [ ]:
import numpy as np 
import pandas as pd
#from autoviz.AutoViz_Class import AutoViz_Class
#AV = AutoViz_Class()
#dft = AV.AutoViz("", sep=',', depVar='resp', dfte=df, header=0, verbose=2,
#                            lowess=False,chart_format='svg',max_rows_analyzed=1500,max_cols_analyzed=30)
import matplotlib.pyplot as plt

def get_most_possible_profit(stock_prices, weight):
    most_possible_profit = 0.0
    for i in range(1,len(stock_prices)):
        if (stock_prices[i-1] < stock_prices[i]):
            most_possible_profit += (stock_prices[i] - stock_prices[i-1])*weight[i]
    return most_possible_profit

def evaluate(datafr):
    p_i = []
    number_of_days = len(pd.unique( datafr.date.to_list() ) )
    for i in range( number_of_days  ):
        current_day_df = datafr[datafr.date == i]
        p_i_current_day = np.array( current_day_df.weight * current_day_df.resp * current_day_df.action ).sum()
        p_i.append(p_i_current_day)
    
    np_p_i = np.array(p_i)
    t = ( np_p_i.sum()/ np.sqrt( (np_p_i * np_p_i).sum() ) ) * np.sqrt(250/number_of_days)
    utility = min(max(0,t), 6 ) * np_p_i.sum()
    return utility
            


df = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv', nrows = 5000)
#df = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
df = df[df.weight>0.0]
df.reset_index(inplace = True, drop = True) 
#df['action'] = np.random.randint(2, size=len(df.index))
df['action'] = np.zeros(len(df.index))

print(df.info )
print(df.head(10))

util = evaluate(df) 
print("Util = ", util)

            
from sklearn.datasets import make_gaussian_quantiles

from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

df.loc[df['resp'] > 0.0, 'action'] = 1
df.loc[df['resp'] <= 0.0, 'action'] = 0
profit_columns = []
loose_columns = []
df = df.dropna()
for (columnName, columnData) in df.iteritems(): 
    if "date" in columnName:
        continue 
    if "resp" in columnName:
        continue 
    if "action" in columnName:
        continue 
    profit_columns.append(df[columnName][df.action > 0.0].to_list())
    loose_columns.append(df[columnName][df.action <= 0.0].to_list())

X1 = np.array ( profit_columns  ) 
y1 = np.array ( df.action[df.action>0.0].to_list()  ) 
X2 = np.array ( loose_columns  ) 
y2 = np.array ( df.action[df.action<=0.0].to_list()  ) 

X1 = X1.transpose()
X2 = X2.transpose()
print("x: " , X1)
print("x shape: " , X1.shape)
print("type: " , type(X1))
print("y: " , y1)
print("y shape: " , y1.shape)
print("type: " , type(y1))


X = np.concatenate((X1, X2))
y = np.concatenate((y1, y2))

# Split data into train and test subsets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, shuffle=True)

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

space = {

            #'units1': hp.quniform('units1', 32,3*1024,1),
            #'dropout1': hp.uniform('dropout1', .00,.50),
            'maxdepth1': hp.uniform('maxdepth1', 1,20),
            'maxdepth2': hp.uniform('maxdepth2', 1,20),
            'maxdepth3': hp.uniform('maxdepth3', 1,20),
            'maxdepth4': hp.uniform('maxdepth4', 1,20),
            #'est1': hp.uniform('est1', 1.0,10000),
            #'est2': hp.uniform('est2', 1.0,10000),
            #'est3': hp.uniform('est3', 1.0,10000),
            'method': hp.uniform('method', 0.0,4.0),
        

        }
def f_nn(params):   
    if int(params['method']) == 0:
        clf = DecisionTreeClassifier(max_depth=int(params['maxdepth1']))
        #clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=int(params['maxdepth2'])),n_estimators=int(params['est1']))
        #clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=int(params['maxdepth3'])),n_estimators=int(params['est2']))
        #clf = BaggingClassifier(DecisionTreeClassifier(max_depth=int(params['maxdepth4'])),n_estimators=int(params['est3']))
        clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=int(params['maxdepth2'])),n_estimators=6000)
        clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=int(params['maxdepth3'])),n_estimators=600)
        clf = BaggingClassifier(DecisionTreeClassifier(max_depth=int(params['maxdepth4'])),n_estimators=100)
    if int(params['method']) == 1:
        clf = GaussianNB()
        clf = AdaBoostClassifier(learning_rate=2.5,base_estimator=GaussianNB(),n_estimators=800,algorithm='SAMME.R') #,algorithm='SAMME.R')
        clf = BaggingClassifier(GaussianNB(),max_samples=0.009,n_estimators=500)
    if int(params['method']) == 2:
        clf = KNeighborsClassifier(n_neighbors=1)
        clf = BaggingClassifier(KNeighborsClassifier(n_neighbors=1),n_estimators=500)
    #if int(params['method']) == 3:
    #    clf = LinearDiscriminantAnalysis()
    #    clf = BaggingClassifier(LinearDiscriminantAnalysis(),max_samples=0.009,n_estimators=10000)
    if int(params['method']) == 3:
        clf = GradientBoostingClassifier(learning_rate=0.1,n_estimators=200)
        clf = RandomForestClassifier(n_estimators=200)
    
       
    clf.fit(X_train, y_train)

    aa = np.mgrid[-100:100:10000]
    Z = clf.predict(np.c_[ aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel() ])                        
    ZZ = clf.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, ZZ)
    auc = roc_auc_score(y_test, ZZ)
    print("AUC: ", auc)

    return {'loss': -auc, 'status': STATUS_OK}

In [ ]:
trials = Trials()
best = fmin(fn=f_nn, space=space, algo=tpe.suggest, max_evals=10, trials=trials)
print('best: ', best)
print('best accuracy: ',-trials.best_trial['result']['loss'])

In [ ]:
# Create and fit a classifier
if int(best['method']) == 0:
    clf = DecisionTreeClassifier(max_depth=int(best['maxdepth1']))
    clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=int(best['maxdepth2'])),n_estimators=6000)
    clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=int(best['maxdepth3'])),n_estimators=600)
    clf = BaggingClassifier(DecisionTreeClassifier(max_depth=int(best['maxdepth4'])),n_estimators=100)
if int(best['method']) == 1:
    clf = GaussianNB()
    clf = AdaBoostClassifier(learning_rate=2.5,base_estimator=GaussianNB(),n_estimators=800,algorithm='SAMME.R') #,algorithm='SAMME.R')
    clf = BaggingClassifier(GaussianNB(),max_samples=0.009,n_estimators=500)
if int(best['method']) == 2:
    clf = KNeighborsClassifier(n_neighbors=1)
    clf = BaggingClassifier(KNeighborsClassifier(n_neighbors=1),n_estimators=10000)
#if int(best['method']) == 3:
#    clf = LinearDiscriminantAnalysis()
#    clf = BaggingClassifier(LinearDiscriminantAnalysis(),max_samples=0.009,n_estimators=int(best['est1']))
if int(best['method']) == 3:
    clf = GradientBoostingClassifier(learning_rate=0.1,n_estimators=200)
    clf = RandomForestClassifier(n_estimators=200)

# Takes extremaly long time
#clf = GaussianProcessClassifier(1.0 * RBF(1.0))
#clf = BaggingClassifier(GaussianProcessClassifier(1.0 * RBF(1.0)))

clf.fit(X_train, y_train)

plot_colors = "br"
plot_step = 0.1
class_names = "AB"
 
plt.figure(figsize=(10, 5))
x_min, x_max = X_test[:, 0].min() - 1, X_test[:, 0].max() + 1
y_min, y_max = X_test[:, 1].min() - 1, X_test[:, 1].max() + 1 
xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                     np.arange(y_min, y_max, plot_step))
aa = np.mgrid[-100:100:10000]
Z = clf.predict(np.c_[ aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel(), aa.ravel() ])
#Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
print("Z ", Z)
print("Z  shape", Z.shape)
#Z = Z.reshape(xx.shape)
print("Z ", Z)
print("Z re-shape", Z.shape)
#cs = plt.contourf(xx, yy, Z, cmap=plt.cm.Paired)
#splt.axis("tight")
 
# Plot the test points

In [ ]:
plt.figure(1,figsize=(10,10))
plt.ylabel("Returns")
plt.xlabel("Time")
plt.title('Stock price returns')
plt.plot(df.resp[0:100])
plt.savefig('/kaggle/working/returns.pdf')
plt.savefig('/kaggle/working/returns.png')

plt.figure(2,figsize=(10,10))
plt.ylabel("Events")
plt.xlabel("Returns")
plt.title('Histogram of returns')
plt.hist(df.resp[0:1000], np.linspace(-0.5,0.5,100))
plt.savefig('/kaggle/working/returns_hist.pdf')
plt.savefig('/kaggle/working/returns_hist.png')

plt.figure(3,figsize=(10,10))
plt.ylabel("Feature_15")
plt.xlabel("Time")
plt.title('Stock Feature_15')
plt.plot(df.feature_15[0:50])
plt.savefig('/kaggle/working/feature_15.pdf')
plt.savefig('/kaggle/working/feature_15.png')

plt.figure(4,figsize=(10,10))
plt.ylabel("Feature_15")
plt.xlabel("Feature_14")
plt.title('Feature_14 vs Feature_15')
plt.scatter(df.feature_14[0:1000],df.feature_15[0:1000])
plt.savefig('/kaggle/working/feature_14_vs_15.pdf')
plt.savefig('/kaggle/working/feature_14_vs_15.png')



In [ ]:
print("Plotting:")
figure_i_min = 100
figure_j_min = 100
figure_i = 120
figure_j = 120
fig, axs = plt.subplots(figure_i-figure_i_min, figure_j-figure_j_min, figsize=(20,20))
for i in range(figure_i-figure_i_min):
    for j in range (figure_j - figure_j_min): 
        for k, n, c in zip(range(2), class_names, plot_colors):
            # plot first 400 test data points
            N = min(400,len(y_test))
            #N = len(y_test)
            idx = np.where(y_test[0:N] == k)
            #print("Plotting ",len(idx)," datapoints., i, j : ", i,j)
            axs[i,j].scatter(X_test[idx, figure_i_min + i], X_test[idx, figure_j_min + j],
                             c=c, cmap=plt.cm.Paired,
                             label= str(figure_i_min + i) + " vs. " + str(figure_j_min + j), alpha=1.0)
            axs[i,j].xaxis.set_ticks([]) 
            axs[i,j].yaxis.set_ticks([]) 
            axs[i,j].xaxis.set_ticklabels([])
            axs[i,j].yaxis.set_ticklabels([])
            #axs[i,j].legend()
fig.tight_layout()

plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig("/kaggle/working/Features_zoomed.pdf")
plt.savefig("/kaggle/working/Features_zoomed.png")
plt.show()


In [ ]:
def GetCorrDf(k = 1, l = 1, Profit=True):
    if Profit:
        return np.corrcoef(df['feature_'+str(k)][df.action > 0.0], df['feature_'+str(l)][df.action > 0.0])[0][1]
    else:
        return np.corrcoef(df['feature_'+str(k)][df.action <= 0.0], df['feature_'+str(l)][df.action <= 0.0])[0][1]
    

#PlotFeatures(4,48)

profit_corr = []
loose_corr = []
for i in range(130):
    profit_row = []
    loose_row = []
    for j in range(130):
        profit_row.append(GetCorrDf(i,j))
        loose_row.append(GetCorrDf(i,j, False))
    loose_corr.append(loose_row)    
    profit_corr.append(profit_row)

print("Done.")
import seaborn as sns; sns.set()
fig, axs = plt.subplots(1, 2, figsize=(25,10))
axs[0] = sns.heatmap(profit_corr,ax=axs[0], vmin=-1.0, vmax=1.0)
axs[0].invert_yaxis()
axs[0].set_title("Profit Heatmap Original Data")
axs[0].set_xlabel("Feature number")
axs[1] = sns.heatmap(loose_corr, ax=axs[1], vmin=-1.0, vmax=1.0)
axs[1].invert_yaxis()
axs[1].set_title("Loose Heatmap Original Data")
axs[1].set_xlabel("Feature number")
plt.savefig("/kaggle/working/Heatmap_df.pdf")
plt.savefig("/kaggle/working/Heatmap_df.png")
plt.show()

def GetCorr(k = 1, l = 1):
    corrcoefs = []
    for m, n, c in zip(range(2), class_names, plot_colors):
            N = len(y_test)
            idx = np.where(y_test[0:N] == m)
            corrcoefs.append(np.corrcoef(X_test[idx, k], X_test[idx, l])[0][1])
    return corrcoefs

#PlotFeatures(4,48)

profit_corr = []
loose_corr = []
for i in range(130):
    profit_row = []
    loose_row = []
    for j in range(130):
        profit_row.append(GetCorr(i,j)[0])
        loose_row.append(GetCorr(i,j)[1])
    loose_corr.append(loose_row)    
    profit_corr.append(profit_row)

print("Done.")
import seaborn as sns; sns.set()
fig, axs = plt.subplots(1, 2, figsize=(25,10))
axs[0] = sns.heatmap(profit_corr,ax=axs[0], vmin=-1.0, vmax=1.0)
axs[0].invert_yaxis()
axs[0].set_title("Profit Heatmap After Classification")
axs[0].set_xlabel("Feature number")
axs[1] = sns.heatmap(loose_corr, ax=axs[1], vmin=-1.0, vmax=1.0)
axs[1].invert_yaxis()
axs[1].set_title("Loose Heatmap After Classification")
axs[1].set_xlabel("Feature number")
plt.savefig("/kaggle/working/Heatmap_classes.pdf")
plt.savefig("/kaggle/working/Heatmap_classes.png")




In [ ]:

# plot the roc curve
figure = plt.figure(figsize=(10, 5))
ZZ = clf.predict_proba(X_test)[:, 1]
print(X_test)
fpr, tpr, _ = roc_curve(y_test, ZZ)
plt.plot(fpr, tpr, label="ROC")
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
#plt.title('ROC curve')
plt.legend(loc='best')
auc = roc_auc_score(y_test, ZZ)
plt.title("Test sample")
plt.text(0.7, 0.5, ('AUC = %.2f' % auc),
        size=15, horizontalalignment='right')
plt.savefig("/kaggle/working/roc.pdf")
plt.savefig("/kaggle/working/roc.png")

plt.show()

In [ ]:
def PlotFeatures(k = 1, l = 1):
    plt.figure(int(str(k)+str(l)) ,figsize = (10,10))
    ax = plt.subplot(111)
    corrcoefs = []
    for m, n, c in zip(range(2), class_names, plot_colors):
            # plot first 400 test data points
            N = min(400,len(y_test))
            #N = len(y_test)
            idx = np.where(y_test[0:N] == m)
            corrcoefs.append(np.corrcoef(X_test[idx, k], X_test[idx, l])[0][1])
            ax.scatter(X_test[idx, k], X_test[idx, l],
                             c=c, cmap=plt.cm.Paired,
                             label="Feature"+str(k) +" vs. "+ str(l), alpha=1.0) 
            ax.set_xlabel('Feature '+str(k))
            ax.set_ylabel('Feature '+str(l))
    plt.savefig(os.path.join(os.getcwd(), 'feature_'+str(k)+'_vs_'+str(l)+'.pdf'))
    plt.show()
    return corrcoefs
PlotFeatures(41,48)